# Session Examples Investigation.
- We sample users with bookings and without bookings. 
- We manually investigate their session activities to get an insight. 

# Summary.
- Among 'action_detail' column seems the most useful in term of interpretability. 
- Users with bookings do not necessarily have a session information about bookings, such as 'request booking'. In other words, session data is not complete.
- For each user, counting each action and use as a 'vector' could be useful.
- To keep sequential information, we can use Markov chain type model. 
- secs_elapsed is 'unusable': they are often extremely big. 
- It might be useful to keep which 'device' they used.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('train_users_2.csv')
df_s = pd.read_csv('sessions.csv')

# Collect user data with sessions.  

In [3]:
ids_with_session = df[df.id.isin(df_s.user_id)].id

In [4]:
df = df[df.country_destination.isin(['NDF', 'US'])]
df = df[df.id.isin(ids_with_session)]

In [5]:
df.shape

(65136, 16)

In [6]:
df_s = df_s[df_s.user_id.isin(ids_with_session)]

In [7]:
df_s.shape

(5537957, 6)

# Sampling

In [8]:
ids_book = df[df.country_destination=='US'].id.sample(5, random_state=0).tolist()
ids_book

['kdtf2ybl8p', '5vxhow36hw', '1dvgg5vpud', 'e3unr37qpy', 'fuj1pyqwu7']

In [9]:
ids_nobook = df[df.country_destination=='NDF'].id.sample(5, random_state=0).tolist()
ids_nobook

['iojqb0tl08', 'wi81v6385f', 'g5xoubt9nh', 'j4cpe9jo8m', 'hv0f4elzam']

In [10]:
df[df.id.isin(ids_book)]

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
150103,1dvgg5vpud,2014-02-06,20140206212716,2014-06-30,MALE,27.0,facebook,0,en,direct,direct,untracked,Web,Windows Desktop,IE,US
155499,e3unr37qpy,2014-02-22,20140222001809,2014-02-23,-unknown-,NaN,basic,0,en,sem-brand,google,omg,Web,iPad,Mobile Safari,US
155987,fuj1pyqwu7,2014-02-23,20140223190956,2015-02-02,FEMALE,31.0,basic,0,en,sem-non-brand,google,omg,Web,Mac Desktop,Firefox,US
179060,kdtf2ybl8p,2014-04-22,20140422224731,2015-04-10,FEMALE,43.0,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari,US
190895,5vxhow36hw,2014-05-18,20140518003125,2014-05-18,MALE,32.0,facebook,0,en,seo,google,linked,Web,Mac Desktop,Chrome,US


In [11]:
df[df.id.isin(ids_nobook)]

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
139762,hv0f4elzam,2014-01-09,20140109172501,NaN,-unknown-,NaN,basic,0,en,content,google,untracked,Web,Mac Desktop,Safari,NDF
140639,g5xoubt9nh,2014-01-11,20140111214016,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Windows Desktop,Chrome,NDF
156571,j4cpe9jo8m,2014-02-25,20140225024819,NaN,MALE,23.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
177520,wi81v6385f,2014-04-19,20140419015036,NaN,MALE,29.0,basic,0,en,direct,direct,untracked,Web,Windows Desktop,Chrome,NDF
203638,iojqb0tl08,2014-06-12,20140612170510,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Mac Desktop,Chrome,NDF


# User activities with booking.

In [12]:
df_s[df_s.user_id == ids_book[0]]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
2707198,kdtf2ybl8p,create,submit,create_user,Mac Desktop,NaN
2707199,kdtf2ybl8p,header_userpic,data,header_userpic,Mac Desktop,1393.0


In [13]:
df_s[df_s.user_id == ids_book[1]]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
3709094,5vxhow36hw,show,view,p3,iPhone,123.0
3709095,5vxhow36hw,social_connections,data,user_social_connections,iPhone,13.0
3709096,5vxhow36hw,reviews,data,listing_reviews,iPhone,10.0
3709097,5vxhow36hw,show,view,user_profile,iPhone,9396.0
3709098,5vxhow36hw,notifications,submit,notifications,iPhone,810.0
3709099,5vxhow36hw,active,-unknown-,-unknown-,-unknown-,640.0
3709100,5vxhow36hw,show,NaN,NaN,iPhone,51.0
3709101,5vxhow36hw,listings,view,user_listings,iPhone,688.0
3709102,5vxhow36hw,active,-unknown-,-unknown-,iPhone,67.0
3709103,5vxhow36hw,campaigns,NaN,NaN,iPhone,1302595.0


In [14]:
df_s[df_s.user_id == ids_book[2]]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
818855,1dvgg5vpud,show,view,p3,Windows Desktop,15100.0
818856,1dvgg5vpud,show,view,user_profile,Windows Desktop,157731.0
818857,1dvgg5vpud,header_userpic,data,header_userpic,Windows Desktop,4124.0
818858,1dvgg5vpud,authenticate,submit,login,Windows Desktop,NaN
818859,1dvgg5vpud,show,NaN,NaN,Windows Desktop,1042.0
818860,1dvgg5vpud,cancellation_policies,view,cancellation_policies,Windows Desktop,57783.0
818861,1dvgg5vpud,calendar_tab_inner2,-unknown-,-unknown-,Windows Desktop,8022.0
818862,1dvgg5vpud,calendar_tab_inner2,-unknown-,-unknown-,Windows Desktop,3468.0
818863,1dvgg5vpud,calendar_tab_inner2,-unknown-,-unknown-,Windows Desktop,6544.0
818864,1dvgg5vpud,calendar_tab_inner2,-unknown-,-unknown-,Windows Desktop,319347.0


In [15]:
df_s[df_s.user_id == ids_book[3]]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
1153725,e3unr37qpy,header_userpic,data,header_userpic,iPad Tablet,7956.0
1153726,e3unr37qpy,search_results,click,view_search_results,iPad Tablet,21218.0
1153727,e3unr37qpy,create,submit,create_user,iPad Tablet,NaN
1153728,e3unr37qpy,signup_login,view,signup_login_page,iPad Tablet,80189.0
1153729,e3unr37qpy,header_userpic,data,header_userpic,iPad Tablet,22641.0
1153730,e3unr37qpy,cancellation_policies,view,cancellation_policies,iPad Tablet,88013.0
1153731,e3unr37qpy,header_userpic,data,header_userpic,iPad Tablet,7775.0
1153732,e3unr37qpy,header_userpic,data,header_userpic,iPad Tablet,158652.0
1153733,e3unr37qpy,index,view,view_search_results,iPad Tablet,6077.0
1153734,e3unr37qpy,lookup,NaN,NaN,iPad Tablet,13950.0


In [16]:
df_s[df_s.user_id == ids_book[4]]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
1186352,fuj1pyqwu7,lookup,NaN,NaN,Mac Desktop,546.0
1186353,fuj1pyqwu7,search_results,click,view_search_results,Mac Desktop,555.0
1186354,fuj1pyqwu7,personalize,data,wishlist_content_update,Mac Desktop,39.0
1186355,fuj1pyqwu7,search_results,click,view_search_results,Mac Desktop,1740.0
1186356,fuj1pyqwu7,index,view,view_search_results,Mac Desktop,3152.0
1186357,fuj1pyqwu7,index,view,view_search_results,Mac Desktop,28539.0
1186358,fuj1pyqwu7,show,NaN,NaN,Mac Desktop,68.0
1186359,fuj1pyqwu7,show,view,p3,Mac Desktop,9873.0
1186360,fuj1pyqwu7,lookup,NaN,NaN,Mac Desktop,556.0
1186361,fuj1pyqwu7,search_results,click,view_search_results,Mac Desktop,3904.0


# User activities without booking.

In [17]:
df_s[df_s.user_id == ids_nobook[0]]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
4765519,iojqb0tl08,update,submit,update_listing,Mac Desktop,21850.0
4765520,iojqb0tl08,update,submit,update_listing,Mac Desktop,6621.0
4765521,iojqb0tl08,update,submit,update_listing,Mac Desktop,15812.0
4765522,iojqb0tl08,index,view,your_listings,Mac Desktop,373.0
4765523,iojqb0tl08,confirm_email,click,confirm_email_link,Mac Desktop,0.0
4765524,iojqb0tl08,confirm_email,click,confirm_email_link,Mac Desktop,32275.0
4765525,iojqb0tl08,update,submit,update_listing,Mac Desktop,30533.0
4765526,iojqb0tl08,update,submit,update_listing,Mac Desktop,27208.0
4765527,iojqb0tl08,header_userpic,data,header_userpic,Mac Desktop,1224.0
4765528,iojqb0tl08,manage_listing,view,manage_listing,Mac Desktop,1649.0


In [18]:
df_s[df_s.user_id == ids_nobook[1]]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
2576589,wi81v6385f,create,submit,create_user,Windows Desktop,NaN
2576590,wi81v6385f,show,view,p3,Windows Desktop,27344.0
2576591,wi81v6385f,ajax_get_referrals_amt,-unknown-,-unknown-,Windows Desktop,89.0
2576592,wi81v6385f,show,view,p1,Windows Desktop,880.0
2576593,wi81v6385f,header_userpic,data,header_userpic,Windows Desktop,1473.0
2576594,wi81v6385f,ajax_referral_banner_experiment_type,-unknown-,-unknown-,Windows Desktop,9.0
2576595,wi81v6385f,show,view,user_profile,Windows Desktop,17438.0
2576596,wi81v6385f,show_personalize,data,user_profile_content_update,Windows Desktop,1574.0
2576597,wi81v6385f,show,view,user_profile,Windows Desktop,419.0
2576598,wi81v6385f,show,view,p3,Windows Desktop,465.0


In [19]:
df_s[df_s.user_id == ids_nobook[2]]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
222792,g5xoubt9nh,lookup,NaN,NaN,Windows Desktop,242.0
222793,g5xoubt9nh,lookup,NaN,NaN,Windows Desktop,331.0
222794,g5xoubt9nh,search_results,click,view_search_results,Windows Desktop,4641.0
222795,g5xoubt9nh,lookup,NaN,NaN,Windows Desktop,320.0
222796,g5xoubt9nh,show,view,p3,Windows Desktop,6166.0
222797,g5xoubt9nh,personalize,data,wishlist_content_update,Windows Desktop,1215.0
222798,g5xoubt9nh,ajax_refresh_subtotal,click,change_trip_characteristics,Windows Desktop,599.0
222799,g5xoubt9nh,similar_listings,data,similar_listings,Windows Desktop,263.0
222800,g5xoubt9nh,show,view,p3,Windows Desktop,34482.0
222801,g5xoubt9nh,show,NaN,NaN,Windows Desktop,1127.0


In [20]:
df_s[df_s.user_id == ids_nobook[3]]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
1224485,j4cpe9jo8m,search_results,click,view_search_results,Mac Desktop,0.0
1224486,j4cpe9jo8m,lookup,NaN,NaN,Mac Desktop,575.0
1224487,j4cpe9jo8m,show,view,p3,Mac Desktop,6872.0
1224488,j4cpe9jo8m,show,NaN,NaN,Mac Desktop,1016.0
1224489,j4cpe9jo8m,personalize,data,wishlist_content_update,Mac Desktop,40.0
1224490,j4cpe9jo8m,similar_listings,data,similar_listings,Mac Desktop,865.0
1224491,j4cpe9jo8m,ajax_refresh_subtotal,click,change_trip_characteristics,Mac Desktop,135.0
1224492,j4cpe9jo8m,ajax_lwlb_contact,click,contact_host,Mac Desktop,65967.0
1224493,j4cpe9jo8m,ajax_refresh_subtotal,click,change_trip_characteristics,Mac Desktop,905.0
1224494,j4cpe9jo8m,ajax_check_dates,click,change_contact_host_dates,Mac Desktop,42.0


In [21]:
df_s[df_s.user_id == ids_nobook[4]]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
172961,hv0f4elzam,personalize,data,wishlist_content_update,Mac Desktop,73.0
172962,hv0f4elzam,show,view,p3,Mac Desktop,55002.0
172963,hv0f4elzam,show,NaN,NaN,Mac Desktop,3172.0
172964,hv0f4elzam,show,view,user_profile,Mac Desktop,95769.0
172965,hv0f4elzam,header_userpic,data,header_userpic,Mac Desktop,830.0
172966,hv0f4elzam,create,submit,create_user,Mac Desktop,NaN
172967,hv0f4elzam,show_personalize,data,user_profile_content_update,Mac Desktop,5439.0
172968,hv0f4elzam,index,view,view_search_results,Mac Desktop,132111.0
172969,hv0f4elzam,lookup,NaN,NaN,Mac Desktop,1402.0
172970,hv0f4elzam,show,view,p3,Mac Desktop,1172.0
